<a href="https://colab.research.google.com/github/saisrikanthmadugula/rkAMM-Simulation-Framework/blob/main/Integrated_Framework_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Appendix: Integrated Simulation Framework Code\n",
    "\n",
    "This notebook contains the complete Python script used to run all simulations presented in the paper, including:\n",
    "\n",
    "1.  **Simulation A: Head-to-Head Ablation Study** (Validates pricing model novelty)\n",
    "2.  **Simulation B: Fat-Tailed Stress Test** (Validates stability and allocation clipping)\n",
    "3.  **Simulation C: Dynamic MAS Model** (Validates the \"virtuous cycle\" and reputation loop)\n",
    "4.  **Simulation D: Esteva et al. Data Validation** (Validates profitability on foundational data)\n",
    "\n",
    "### Step 1: Install Required Libraries"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "install-libs"
   },
   "outputs": [],
   "source": [
    "!pip install requests pandas numpy matplotlib seaborn"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Step 2: Import Libraries"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "imports"
   },
   "outputs": [],
   "source": [
    "import numpy as np\n",
    "import pandas as pd\n",
    "import sys\n",
    "import requests\n",
    "import io\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "from matplotlib.ticker import FuncFormatter"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Step 3: Define Core Agent Classes"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {
    "id": "definitions-agents"
   },
   "outputs": [],
   "source": [
    "# ==============================================================================\n",
    "# --- AGENT DEFINITIONS (The Core Framework) ---\n",
    "# ==============================================================================\n",
    "\n",
    "class RiskAssessmentAgent:\n",
    "    \"\"\" AGENT 1: The STATEFUL AI Agent (Simulated) \"\"\"\n",
    "    def __init__(self, initial_borrowers):\n",
    "        # Initialize all borrowers with a \"newer\" profile\n",
    "        self.borrower_reputations = {\n",
    "            borrower_id: np.clip(np.random.normal(loc=45, scale=10), 10, 80)\n",
    "            for borrower_id in initial_borrowers\n",
    "        }\n",
    "        # Constants for dynamic updates\n",
    "        self.REPAY_BUMP = 3\n",
    "        self.DEFAULT_PENALTY = -15\n",
    "        self.MAX_COLLATERAL = 0.8\n",
    "        self.MIN_P_D = 0.01\n",
    "        self.MAX_P_D = 0.80\n",
    "\n",
    "    def assess_loan_terms(self, borrower_id):\n",
    "        \"\"\" Calculates PD and C based on the agent's current reputation \"\"\"\n",
    "        reputation = self.borrower_reputations.get(borrower_id, 45.0) # Default new borrowers\n",
    "        p_d = self.MAX_P_D - (reputation / 100) * (self.MAX_P_D - self.MIN_P_D)\n",
    "        collateral_level = self.MAX_COLLATERAL * (1.0 - (reputation / 100))\n",
    "        return np.clip(p_d, self.MIN_P_D, self.MAX_P_D), np.clip(collateral_level, 0, self.MAX_COLLATERAL)\n",
    "\n",
    "    def update_reputation(self, borrower_id, outcome):\n",
    "        \"\"\" The reputation feedback loop \"\"\"\n",
    "        if outcome == \"Repaid\":\n",
    "            self.borrower_reputations[borrower_id] += self.REPAY_BUMP\n",
    "        else: # Default\n",
    "            self.borrower_reputations[borrower_id] += self.DEFAULT_PENALTY\n",
    "        self.borrower_reputations[borrower_id] = np.clip(\n",
    "            self.borrower_reputations[borrower_id], 0, 100\n",
    "        )\n",
    "\n",
    "class PricingAgent:\n",
    "    \"\"\" AGENT 2: The rkAMM Pricing Engine (On-Chain Logic) \"\"\"\n",
    "    def __init__(self):\n",
    "        pass\n",
    "\n",
    "    def calculate_premium(self, p_d, collateral_level, loan_amount=1.0):\n",
    "        \"\"\" The Reverse Kelly Premium Formula (q) \"\"\"\n",
    "        L = loan_amount * (1.0 - collateral_level) # Loss Given Default\n",
    "        # Handle edge cases\n",
    "        if L <= 0 or p_d <= 0: return 0.0, L\n",
    "        if p_d >= 1.0: return np.inf, L\n",
    "        # Calculate premium\n",
    "        premium = (L * p_d) / (1.0 - p_d)\n",
    "        return premium, L\n",
    "\n",
    "    def calculate_allocation_fraction(self, p_d, collateral_level, premium, loan_amount=1.0):\n",
    "        \"\"\" The Kelly Allocation Formula (f*) \"\"\"\n",
    "        if premium <= 0 or loan_amount <= 0:\n",
    "            return 0.0\n",
    "        \n",
    "        # L = Loss Given Default\n",
    "        L = loan_amount * (1.0 - collateral_level)\n",
    "        if L <= 0:\n",
    "             # If fully collateralized, any premium is profit, allocation is \"safe\".\n",
    "            return 1.0 \n",
    "            \n",
    "        # q_ratio is the net odds (premium / amount_at_risk)\n",
    "        q_ratio = premium / L\n",
    "        \n",
    "        # f* = (q(1-p) - p) / q \n",
    "        # (where p=p_d and q=q_ratio)\n",
    "        numerator = (q_ratio * (1.0 - p_d)) - p_d\n",
    "        \n",
    "        if numerator <= 0:\n",
    "            return 0.0 # Clip allocation, loan is unprofitable\n",
    "        \n",
    "        # The fraction f* is (numerator / q_ratio). We cap it at 1 (100% of pool)\n",
    "        return min(1.0, numerator / q_ratio)\n",
    "\n",
    "class BlockchainEnforcementAgent:\n",
    "    \"\"\" AGENT 3: The Blockchain Simulator \"\"\"\n",
    "    def __init__(self):\n",
    "        pass\n",
    "\n",
    "    def execute_loan(self, p_d):\n",
    "        \"\"\" Simulates the binary outcome of the loan \"\"\"\n",
    "        return \"Repaid\" if np.random.rand() >= p_d else \"Default\""
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Step 4: Define Simulation Functions"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "def run_ablation_study(n_loans=10000, n_trials=50):\n",
    "    \"\"\" SIMULATION A: Head-to-Head Ablation Study (for Table 1) \"\"\"\n",
    "    print(f\"--- Running Ablation Study (Head-to-Head, {n_trials} trials) ---\")\n",
    "    \n",
    "    strategies = [\"Reverse Kelly\", \"Proportional Premium\", \"Fixed Premium\"]\n",
    "    all_results = {s: [] for s in strategies}\n",
    "    \n",
    "    for trial in range(n_trials):\n",
    "        sys.stdout.write(f\"\\r  Running trial {trial + 1}/{n_trials}...\")\n",
    "        sys.stdout.flush()\n",
    "        \n",
    "        # Generate a fixed stream of 10,000 loans for this trial\n",
    "        loan_stream = []\n",
    "        for _ in range(n_loans):\n",
    "            loan_stream.append({\n",
    "                'p_d': np.random.beta(2, 18), # Avg default rate ~10%\n",
    "                'c': np.random.uniform(0.0, 0.6) # Collateral 0-60%\n",
    "            })\n",
    "\n",
    "        pricing_agent = PricingAgent()\n",
    "        blockchain_agent = BlockchainEnforcementAgent()\n",
    "\n",
    "        for strategy in strategies:\n",
    "            pool_value = 10000.0 # Reset pool for each strategy\n",
    "            capital_deployed = 0\n",
    "            total_possible_capital = 0\n",
    "            loan_amount = 100.0 # Fixed loan size\n",
    "            drawdowns = []\n",
    "            peak_value = pool_value\n",
    "            losses = 0\n",
    "\n",
    "            for loan in loan_stream:\n",
    "                p_d, c = loan['p_d'], loan['c']\n",
    "                premium, loss = 0.0, loan_amount * (1.0 - c)\n",
    "                allocation_fraction = 0.0\n",
    "\n",
    "                if strategy == \"Reverse Kelly\":\n",
    "                    premium, _ = pricing_agent.calculate_premium(p_d, c, loan_amount)\n",
    "                    allocation_fraction = pricing_agent.calculate_allocation_fraction(p_d, c, premium, loan_amount)\n",
    "                \n",
    "                elif strategy == \"Proportional Premium\":\n",
    "                    premium = (2 * p_d * (1.0 - c)) * loan_amount # Premium = 2 * Expected Loss\n",
    "                    allocation_fraction = 1.0 # Always deploy\n",
    "                \n",
    "                elif strategy == \"Fixed Premium\":\n",
    "                    premium = (0.10 / 12) * loan_amount # 10% APR, monthly\n",
    "                    allocation_fraction = 1.0 # Always deploy\n",
    "                \n",
    "                allocation = 0.0\n",
    "                if allocation_fraction > 0:\n",
    "                    allocation = loan_amount # Fund the loan\n",
    "                \n",
    "                if allocation > 0 and pool_value >= allocation:\n",
    "                    capital_deployed += allocation\n",
    "                    total_possible_capital += loan_amount\n",
    "                    outcome = blockchain_agent.execute_loan(p_d)\n",
    "                    \n",
    "                    if outcome == \"Repaid\":\n",
    "                        pool_value += premium\n",
    "                    else: # Default\n",
    "                        losses += 1\n",
    "                        pool_value -= loss\n",
    "                \n",
    "                # Update drawdown\n",
    "                peak_value = max(peak_value, pool_value)\n",
    "                drawdowns.append((peak_value - pool_value) / peak_value if peak_value > 0 else 0)\n",
    "\n",
    "            # Store metrics for this trial\n",
    "            final_cagr = ((pool_value / 10000.0)**(1 / (n_loans/1000)) - 1) if pool_value > 0 else -1.0 # Simple CAGR approx.\n",
    "            all_results[strategy].append({\n",
    "                'cagr': final_cagr,\n",
    "                'max_drawdown': max(drawdowns),\n",
    "                'loss_ratio': losses / n_loans if n_loans > 0 else 0,\n",
    "                'capital_utilization': capital_deployed / total_possible_capital if total_possible_capital > 0 else 0\n",
    "            })\n",
    "    \n",
    "    print(\"\\nAblation study complete.\")\n",
    "    \n",
    "    # Calculate and print final mean/CI for Table 1\n",
    "    print(\"\\n--- Ablation Study Results (Table 1) ---\")\n",
    "    for strategy in strategies:\n",
    "        df = pd.DataFrame(all_results[strategy])\n",
    "        mean = df.mean()\n",
    "        ci = 1.96 * df.std() / np.sqrt(n_trials) # 95% CI\n",
    "        print(f\"\\nStrategy: {strategy}\")\n",
    "        print(f\"  CAGR: {mean['cagr']:.1%} (±{ci['cagr']:.1%})\")\n",
    "        print(f\"  Max Drawdown: {mean['max_drawdown']:.1%} (±{ci['max_drawdown']:.1%})\")\n",
    "        print(f\"  Loss Ratio: {mean['loss_ratio']:.1%} (±{ci['loss_ratio']:.1%})\")\n",
    "        print(f\"  Capital Utilization: {mean['capital_utilization']:.1%} (±{ci['capital_utilization']:.1%})\")\n",
    "\n",
    "def run_stress_test(n_loans=100):\n",
    "    \"\"\" SIMULATION B: Fat-Tailed Stress Test \"\"\"\n",
    "    print(f\"\\n--- Running Fat-Tailed PD Shock Stress Test ---\")\n",
    "    \n",
    "    # Generate a stream of loans, some with fat-tailed PDs\n",
    "    loan_stream = []\n",
    "    for _ in range(n_loans):\n",
    "        # 90% chance of normal PD, 10% chance of extreme PD\n",
    "        if np.random.rand() < 0.9:\n",
    "            p_d = np.random.beta(2, 18) # Normal PD\n",
    "        else:\n",
    "            p_d = np.random.uniform(0.4, 0.8) # Extreme PD\n",
    "        \n",
    "        loan_stream.append({\n",
    "            'p_d': p_d,\n",
    "            'c': np.random.uniform(0.0, 0.3) # Low collateral\n",
    "        })\n",
    "\n",
    "    pricing_agent = PricingAgent()\n",
    "    print(\"PD Stream | Allocation (f*)\")\n",
    "    print(\"----------------------------\")\n",
    "    for loan in loan_stream:\n",
    "        p_d, c = loan['p_d'], loan['c']\n",
    "        premium, _ = pricing_agent.calculate_premium(p_d, c)\n",
    "        allocation_fraction = pricing_agent.calculate_allocation_fraction(p_d, c, premium)\n",
    "        \n",
    "        if p_d > 0.4: # Only print the shock events\n",
    "            print(f\"PD: {p_d:.2f}, c: {c:.2f} | Premium: {premium:.2f} | f*: {allocation_fraction:.2f} {'(CLIPPED)' if allocation_fraction == 0.0 else ''}\")\n",
    "    print(\"Stress test complete. Note how high PDs result in f* = 0.0.\")\n",
    "\n",
    "def run_dynamic_mas_simulation(n_borrowers=500, loans_per_borrower=20):\n",
    "    \"\"\" SIMULATION C: Dynamic MAS (for 'Virtuous Cycle') \"\"\"\n",
    "    print(f\"\\n--- Running Dynamic MAS Simulation ('Virtuous Cycle') ---\")\n",
    "    borrower_ids = list(range(n_borrowers))\n",
    "    risk_agent = RiskAssessmentAgent(borrower_ids)\n",
    "    pricing_agent = PricingAgent()\n",
    "    blockchain_agent = BlockchainEnforcementAgent()\n",
    "    results = []\n",
    "    \n",
    "    for loan_num in range(loans_per_borrower):\n",
    "        sys.stdout.write(f\"\\r  Processing MAS loan round {loan_num + 1}/{loans_per_borrower}...\")\n",
    "        sys.stdout.flush()\n",
    "        \n",
    "        for borrower_id in borrower_ids:\n",
    "            current_rep = risk_agent.borrower_reputations.get(borrower_id)\n",
    "            p_d, collateral_level = risk_agent.assess_loan_terms(borrower_id)\n",
    "            premium, _ = pricing_agent.calculate_premium(p_d, collateral_level)\n",
    "            outcome = blockchain_agent.execute_loan(p_d)\n",
    "            risk_agent.update_reputation(borrower_id, outcome)\n",
    "            results.append({\n",
    "                \"loan_round\": loan_num,\n",
    "                \"borrower_id\": borrower_id,\n",
    "                \"initial_reputation\": current_rep,\n",
    "                \"p_d\": p_d, # Store p_d\n",
    "                \"collateral_level\": collateral_level, # Store collateral\n",
    "                \"outcome\": outcome, # Store outcome\n",
    "                \"final_reputation\": risk_agent.borrower_reputations.get(borrower_id)\n",
    "            })\n",
    "            \n",
    "    print(\"\\nDynamic MAS simulation complete.\")\n",
    "    df = pd.DataFrame(results)\n",
    "    \n",
    "    # --- Analysis for Dynamic MAS ---\n",
    "    \n",
    "    # 1. Bimodal Distribution (Figure 3)\n",
    "    final_reputations = df[df['loan_round'] == loans_per_borrower - 1]['final_reputation']\n",
    "    print(f\"Mean final reputation: {final_reputations.mean():.2f} (Std: {final_reputations.std():.2f})\")\n",
    "    print(\"Bimodal distribution is evident in the histogram plot (Figure 3).\")\n",
    "\n",
    "    # 2. Collateral Reduction\n",
    "    avg_collateral_per_round = df.groupby('loan_round')['collateral_level'].mean()\n",
    "    \n",
    "    df_repaid = df[df['outcome'] == 'Repaid'].copy()\n",
    "    df_repaid['next_collateral'] = risk_agent.MAX_COLLATERAL * (1.0 - (df_repaid['final_reputation'] / 100))\n",
    "    df_repaid['collateral_change'] = df_repaid['next_collateral'] - df_repaid['collateral_level']\n",
    "    avg_collateral_change = df_repaid['collateral_change'].mean()\n",
    "    print(f\"Avg. collateral change after 1 successful repayment: {avg_collateral_change*100:.2f}%\")\n",
    "    \n",
    "    # --- Plotting for Dynamic MAS ---\n",
    "    plt.figure(figsize=(14, 6))\n",
    "    plt.subplot(1, 2, 1)\n",
    "    sns.histplot(final_reputations, bins=20, kde=True)\n",
    "    plt.title('Final Reputation Score Distribution (Fig. 3)', fontsize=14)\n",
    "    plt.xlabel('Reputation Score', fontsize=12)\n",
    "    plt.ylabel('Number of Borrowers', fontsize=12)\n",
    "\n",
    "    plt.subplot(1, 2, 2)\n",
    "    avg_collateral_per_round.plot(marker='o')\n",
    "    plt.title('Avg. Required Collateral Over Time', fontsize=14)\n",
    "    plt.xlabel('Loan Round', fontsize=12)\n",
    "    plt.ylabel('Average Collateral Level', fontsize=12)\n",
    "    plt.gca().yaxis.set_major_formatter(FuncFormatter('{:.1%}'.format))\n",
    "    plt.grid(True, linestyle='--', alpha=0.6)\n",
    "    \n",
    "    plt.tight_layout()\n",
    "    plt.show()\n",
    "\n",
    "def run_esteva_data_validation():\n",
    "    \"\"\" SIMULATION D: Esteva et al. Data Validation (for Table 2) \"\"\"\n",
    "    print(f\"\\n--- Running Esteva et al. (2023) Data Validation ---\")\n",
    "    DATA_URL = \"https://raw.githubusercontent.com/ballesterosbr/rkAMM/master/notebooks/simulated/l_invoices_10k_0.6_0.3_0.01.csv\"\n",
    "    \n",
    "    try:\n",
    "        print(f\"Downloading dataset from: {DATA_URL}\")\n",
    "        r = requests.get(DATA_URL)\n",
    "        r.raise_for_status()\n",
    "        df_invoices = pd.read_csv(io.StringIO(r.text))\n",
    "        print(f\"Successfully loaded {len(df_invoices)} invoices from GitHub.\")\n",
    "        print(\"Dataset 'p_d' distribution (as per Esteva et al.):\")\n",
    "        print(df_invoices['p_d'].value_counts(normalize=True))\n",
    "        print(\"\\n\" + \"=\"*50 + \"\\n\")\n",
    "        \n",
    "        pricing_agent = PricingAgent()\n",
    "        blockchain_agent = BlockchainEnforcementAgent()\n",
    "        \n",
    "        scenarios = [0.0, 0.6] # 0% and 60% collateral\n",
    "        summary = {}\n",
    "\n",
    "        for c_level in scenarios:\n",
    "            print(f\"  Running Esteva validation with C = {c_level*100:.0f}%...\")\n",
    "            net_profit = 0\n",
    "            total_premiums = 0\n",
    "            total_losses = 0\n",
    "            defaults = 0\n",
    "            \n",
    "            for i, row in df_invoices.iterrows():\n",
    "                p_d = row['p_d']\n",
    "                premium, loss = pricing_agent.calculate_premium(p_d, c_level, loan_amount=1.0)\n",
    "                outcome = blockchain_agent.execute_loan(p_d)\n",
    "                \n",
    "                total_premiums += premium\n",
    "                if outcome == \"Default\":\n",
    "                    defaults += 1\n",
    "                    total_losses += loss\n",
    "                    net_profit -= loss\n",
    "                \n",
    "                net_profit += premium\n",
    "                \n",
    "                if (i + 1) % 1000 == 0:\n",
    "                    sys.stdout.write(f\"\\r    ...processed {i+1}/{len(df_invoices)} Esteva loans.\")\n",
    "                    sys.stdout.flush()\n",
    "            \n",
    "            print(\"\\n  ...Run complete.\")\n",
    "            summary[f\"Scenario C={c_level}\"] = {\n",
    "                \"Total Premiums\": total_premiums,\n",
    "                \"Total Defaults\": defaults,\n",
    "                \"Total Losses\": total_losses,\n",
    "                \"Net Profit\": net_profit\n",
    "            }\n",
    "            \n",
    "        print(\"\\nEsteva data validation complete.\")\n",
    "        print(\"\\n--- Esteva Validation Results (Table 2) ---\")\n",
    "        # Use .T to transpose for better readability\n",
    "        print(pd.DataFrame(summary).T.to_markdown(floatfmt=\",.2f\"))\n",
    "        \n",
    "    except Exception as e:\n",
    "        print(f\"Error: Could not download or run Esteva validation. {e}\")\n",
    "        print(\"Please ensure you have an active internet connection.\")"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "### Step 5: Run All Simulations"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "# Run Simulation A (Ablation Study)\n",
    "run_ablation_study(n_loans=10000, n_trials=50)\n",
    "\n",
    "print(\"\\n\" + \"=\"*50 + \"\\n\")\n",
    "\n",
    "# Run Simulation B (Stress Test)\n",
    "run_stress_test(n_loans=20)\n",
    "\n",
    "print(\"\\n\" + \"=\"*50 + \"\\n\")\n",
    "\n",
    "# Run Simulation C (Dynamic MAS)\n",
    "run_dynamic_mas_simulation(n_borrowers=500, loans_per_borrower=20)\n",
    "\n",
    "print(\"\\n\" + \"=\"*50 + \"\\n\")\n",
    "\n",
    "# Run Simulation D (Esteva Data Validation)\n",
    "run_esteva_data_validation()"
   ]
  }
 ],
 "metadata": {
  "colab": {
   "provenance": []
  },
  "kernelspec": {
   "display_name": "Python 3",
   "name": "python3"
  },
  "language_info": {
   "name": "python"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 0
}

SyntaxError: invalid syntax (ipython-input-598862961.py, line 163)